# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# NetCDF Exports

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

### Load the base watershed model
  - We will use the base watershed model to demonstrate `MODFLOW 6` `NetCDF` export functionality  

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

### Update model to save structured `NetCDF` output
  - Set `start_date_time` in the TDIS file
  - Define `EXPORT_NETCDF` variable in model name file and set to `structured` output type

In [ ]:
sim.tdis.start_date_time = "1980-01-01"

In [ ]:
gwf = sim.get_model()
gwf.name_file.export_netcdf = "structured"

### Run the model
  - Change simulation workspace
  - Write model files and run the model

In [ ]:
ws = pl.Path("temp/step0/structured")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Visualize export data
  - Load the structured NetCDF file with `xarray`
  - Visualize `head` layer 1, timestep 1

In [ ]:
pl.Path(ws / "model.nc").is_file()

In [ ]:
nc_ds = xa.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["head"].isel(time=0, z=0).plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

### Modify the model to save mesh NetCDF output
  - Update the `export_netcdf` variable in the model name file to `ugrid` to create mesh NetCDF output

In [ ]:
gwf.name_file.export_netcdf = "ugrid"

### Run the model
  - Change simulation workspace
  - Write model files and run the model

In [ ]:
ws = pl.Path("temp/step0/mesh")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Visualize export data
  - Load the mesh NetCDF file with `xugrid`
  - Visualize `head` layer 1, timestep 1

In [ ]:
nc_ds = xu.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["head_l1"].isel(time=0).ugrid.plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

### Create cross-sections using `xugrid`
  - Create a cross-section along a column (y-direction)

In [ ]:
section = nc_ds.ugrid.sel(x=101250)

In [ ]:
section

In [ ]:
section["head_l1"].isel(time=0).plot.line(x="mesh_y")  # , where="mid")
plt.show(block=False)

  - Create a cross-section along a row (x-direction)

In [ ]:
section = nc_ds.ugrid.sel(y=41250)
section["head_l1"].isel(time=0).plot.step(x="mesh_x", where="mid")
plt.show(block=False)